# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('eo_merged_features.npz') 

# Access the concatenated features from the .npz file
X_merged_features = data['X_merged_features']

# Close the loaded file
data.close()

#print the merged array
X_merged_features

array([[[-5.21146963e-05, -5.10212700e-05, -5.48470268e-05, ...,
          3.26074783e-11,  4.63140062e-01,  1.63413091e+00],
        [-9.05187354e-06, -7.71941472e-06, -1.01702073e-05, ...,
          2.75444251e-11,  4.68105779e-01,  1.54852559e+00],
        [ 2.67863504e-05,  2.69096507e-05,  2.54325553e-05, ...,
          2.74209376e-11,  3.65731034e-01,  1.72778820e+00],
        ...,
        [ 9.28187476e-07,  3.35509979e-07,  1.02039589e-06, ...,
          2.00323483e-11,  3.09365829e-01,  2.04954932e+00],
        [ 3.27318188e-05,  3.21226205e-05,  3.36029402e-05, ...,
          1.35657944e-11,  3.85630195e-01,  1.60045474e+00],
        [ 7.80324226e-06,  8.28110033e-06,  9.39076706e-06, ...,
          4.32161140e-11,  3.63171004e-01,  1.89534486e+00]],

       [[ 2.28545727e-05,  2.31652880e-05,  2.10154211e-05, ...,
          2.38972234e-11,  5.33616949e-01,  1.49649669e+00],
        [ 3.05036576e-05,  2.91500290e-05,  3.23404999e-05, ...,
          2.46533063e-11,  5.29122302e

In [3]:
data = np.load('eyes_open_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('eyes_open_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
Y=Y
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_merged_features.shape, Y.shape, groups_array.shape)

(18145, 20, 238) (18145,) (18145,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
import os
import csv

metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_merged_features, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_merged_features[train_index], X_merged_features[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

    # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
64/64 [==============================] - 11s 58ms/step - loss: 0.3696 - accuracy: 0.8347 - val_loss: 0.3398 - val_accuracy: 0.8562
Epoch 2/200
64/64 [==============================] - 2s 34ms/step - loss: 0.2022 - accuracy: 0.9236 - val_loss: 0.2594 - val_accuracy: 0.8926
Epoch 3/200
64/64 [==============================] - 2s 34ms/step - loss: 0.1372 - accuracy: 0.9477 - val_loss: 0.1964 - val_accuracy: 0.9289
Epoch 4/200
64/64 [==============================] - 2s 31ms/step - loss: 0.1035 - accuracy: 0.9596 - val_loss: 0.1389 - val_accuracy: 0.9504
Epoch 5/200
64/64 [==============================] - 2s 31ms/step - loss: 0.0834 - accuracy: 0.9677 - val_loss: 0.1173 - val_accuracy: 0.9642
Epoch 6/200
64/64 [==============================] - 2s 31ms/step - loss: 0.0647 - accuracy: 0.9763 - val_loss: 0.1222 - val_accuracy: 0.9691
Epoch 7/200
64/64 [==============================] - 2s 29ms/step - loss: 0.0562 - accuracy: 0.9794 - val_loss: 0.1127 - val_accuracy: 0.969

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9862,0.9853,0.9884,0.9868,0.9861,0.9839,0.9884,0.9724,0.9724
1,0.9846,0.9883,0.9820,0.9852,0.9847,0.9873,0.9820,0.9691,0.9691
2,0.9873,0.9904,0.9852,0.9878,0.9874,0.9896,0.9852,0.9746,0.9746
3,0.9818,0.9790,0.9863,0.9826,0.9816,0.9770,0.9863,0.9636,0.9636
4,0.9851,0.9883,0.9831,0.9857,0.9852,0.9873,0.9831,0.9702,0.9702
5,0.9840,0.9862,0.9831,0.9846,0.9841,0.9850,0.9831,0.9680,0.9680
6,0.9862,0.9832,0.9905,0.9868,0.9860,0.9816,0.9905,0.9724,0.9724
7,0.9835,0.9811,0.9873,0.9842,0.9833,0.9792,0.9873,0.9668,0.9669
8,0.9774,0.9758,0.9810,0.9784,0.9772,0.9735,0.9810,0.9547,0.9547
9,0.9862,0.9873,0.9863,0.9868,0.9862,0.9862,0.9863,0.9724,0.9724


In [13]:
metrics_df.round(4).to_csv('COMBINED_EO_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("COMBINED_EO_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9862,0.9853,0.9884,0.9868,0.9861,0.9839,0.9884,0.9724,0.9724
1,0.9846,0.9883,0.9820,0.9852,0.9847,0.9873,0.9820,0.9691,0.9691
2,0.9873,0.9904,0.9852,0.9878,0.9874,0.9896,0.9852,0.9746,0.9746
3,0.9818,0.9790,0.9863,0.9826,0.9816,0.9770,0.9863,0.9636,0.9636
4,0.9851,0.9883,0.9831,0.9857,0.9852,0.9873,0.9831,0.9702,0.9702
5,0.9840,0.9862,0.9831,0.9846,0.9841,0.9850,0.9831,0.9680,0.9680
6,0.9862,0.9832,0.9905,0.9868,0.9860,0.9816,0.9905,0.9724,0.9724
7,0.9835,0.9811,0.9873,0.9842,0.9833,0.9792,0.9873,0.9668,0.9669
8,0.9774,0.9758,0.9810,0.9784,0.9772,0.9735,0.9810,0.9547,0.9547
9,0.9862,0.9873,0.9863,0.9868,0.9862,0.9862,0.9863,0.9724,0.9724


In [16]:
(result.mean()*100).round(2)

Accuracy       98.42
Precision      98.45
Recall         98.53
F1             98.49
Sensitivity    98.42
Specificity    98.31
ROC_AUC        98.53
Kappa          96.84
MCC            96.84
dtype: float64

In [17]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])